In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import warnings
import xgboost as xgb
import lightgbm as lgb
from scipy.stats import skew
from scipy import stats
from scipy.stats.stats import pearsonr
from scipy.stats import norm
from collections import Counter
from sklearn.linear_model import LinearRegression,LassoCV, Ridge, LassoLarsCV,ElasticNetCV
from sklearn.model_selection import GridSearchCV, cross_val_score, learning_curve
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, Normalizer, RobustScaler
warnings.filterwarnings('ignore')
sns.set(style='white', context='notebook', palette='deep')
%config InlineBackend.figure_format = 'retina' #set 'png' here when working on notebook
%matplotlib inline
from math import sqrt
from numpy import loadtxt
from itertools import product
from tqdm import tqdm
from sklearn import preprocessing
from xgboost import plot_tree
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import gc

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

kernel_with_output = True

In [4]:
# Load train and Test set
sales_train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
shop= pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
item_cat = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")

In [5]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [6]:
sales_train = reduce_mem_usage(sales_train)
test = reduce_mem_usage(test)
shop = reduce_mem_usage(shop)
items = reduce_mem_usage(items)
item_cat = reduce_mem_usage(item_cat)

Mem. usage decreased to 50.40 Mb (62.5% reduction)
Mem. usage decreased to  1.43 Mb (70.8% reduction)
Mem. usage decreased to  0.00 Mb (38.6% reduction)
Mem. usage decreased to  0.23 Mb (54.2% reduction)
Mem. usage decreased to  0.00 Mb (39.9% reduction)


In [7]:
if kernel_with_output:
    # For every month we create a grid from all shops/items combinations from that month
    grid = []
    for block_num in sales_train['date_block_num'].unique():
        cur_shops = sales_train[sales_train['date_block_num']==block_num]['shop_id'].unique()
        cur_items = sales_train[sales_train['date_block_num']==block_num]['item_id'].unique()
        grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
    index_cols = ['shop_id', 'item_id', 'date_block_num']
    grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

    # Aggregations
    sales_train['item_cnt_day'] = sales_train['item_cnt_day'].clip(0,20)
    groups = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'])
    trainset = groups.agg({'item_cnt_day':'sum', 'item_price':'mean'}).reset_index()
    trainset = trainset.rename(columns = {'item_cnt_day' : 'item_cnt_month'})
    trainset['item_cnt_month'] = trainset['item_cnt_month'].clip(0,20)

    trainset = pd.merge(grid,trainset,how='left',on=index_cols)
    trainset.item_cnt_month = trainset.item_cnt_month.fillna(0)

    # Get category id
    trainset = pd.merge(trainset, items[['item_id', 'item_category_id']], on = 'item_id')
    trainset.to_csv('trainset_with_grid.csv')

    trainset.head()

In [8]:
#if kernel_with_output:
    #prev_month_selector = (trainset.date_block_num==33)
    #train_subset = trainset[prev_month_selector]
   # groups = train_subset[['shop_id', 'item_id', 'item_cnt_month']].groupby(by = ['shop_id', 'item_id'])
   # train_subset = groups.agg({'item_cnt_month':'sum'}).reset_index()
   # train_subset.head(3)

In [9]:
#if kernel_with_output:
   # merged = test.merge(train_subset, on=["shop_id", "item_id"], how="left")[["ID", "item_cnt_month"]]
    #merged.isna().sum()

In [10]:
gc.collect()

64

In [11]:
#if kernel_with_output:
   # merged['item_cnt_month'] = merged.item_cnt_month.fillna(0).clip(0,20)
   # submission = merged.set_index('ID')
   # submission.to_csv('benchmark.csv')

In [12]:
#del merged
#del train_subset


In [13]:
gc.collect()

60

In [14]:
if kernel_with_output:
    # Extract features and target we want
    baseline_features = ['shop_id', 'item_id', 'item_category_id', 'date_block_num', 'item_cnt_month']
    train = trainset[baseline_features]
    # Remove pandas index column
    train = train.set_index('shop_id')
    train.item_cnt_month = train.item_cnt_month.astype(int)
    train['item_cnt_month'] = train.item_cnt_month.fillna(0).clip(0,20)
    # Save train set to file
    train.to_csv('train.csv')

In [15]:
if kernel_with_output:
    # Set seeds and options
    np.random.seed(10)
    pd.set_option('display.max_rows', 231)
    pd.set_option('display.max_columns', 100)

    # Feature engineering list
    new_features = []
    enable_feature_idea = [True, True, True, True, True, True, True, True, True, True]

    # Some parameters(maybe add more periods, score will be better) [1,2,3,12]
    lookback_range = [1,2,3,4,5,6,7,8,9,10,11,12]

    tqdm.pandas()


In [16]:
gc.collect()

75

In [17]:
if kernel_with_output:
    current = time.time()

    trainset = pd.read_csv('trainset_with_grid.csv')
    items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
    shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')


    # Only use more recent data
    start_month = 0
    end_month = 33
    trainset = trainset[['shop_id', 'item_id', 'item_category_id', 'date_block_num', 'item_price', 'item_cnt_month']]
    trainset = trainset[(trainset.date_block_num >= start_month) & (trainset.date_block_num <= end_month)]

    print('Loading test set...')
    test_dataset = loadtxt('/kaggle/input/competitive-data-science-predict-future-sales/test.csv', delimiter="," ,skiprows=1, usecols = (1,2), dtype=int)
    testset = pd.DataFrame(test_dataset, columns = ['shop_id', 'item_id'])

    print('Merging with other datasets...')
    # Get item category id into test_df
    testset = testset.merge(items[['item_id', 'item_category_id']], on = 'item_id', how = 'left')
    testset['date_block_num'] = 34
    # Make testset contains same column as trainset so we can concatenate them row-wise
    testset['item_cnt_month'] = -1

    train_test_set = pd.concat([trainset, testset], axis = 0) 

    end = time.time()
    diff = end - current
    print('Took ' + str(int(diff)) + ' seconds to train and predict val set')

Loading test set...
Merging with other datasets...
Took 10 seconds to train and predict val set


In [18]:
del end_month
del start_month
gc.collect()

20

In [19]:
if kernel_with_output:
    item_cat = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')

    # Fix category
    l_cat = list(item_cat.item_category_name)
    for ind in range(0,1):
        l_cat[ind] = 'PC Headsets / Headphones'
    for ind in range(1,8):
        l_cat[ind] = 'Access'
    l_cat[8] = 'Tickets (figure)'
    l_cat[9] = 'Delivery of goods'
    for ind in range(10,18):
        l_cat[ind] = 'Consoles'
    for ind in range(18,25):
        l_cat[ind] = 'Consoles Games'
    l_cat[25] = 'Accessories for games'
    for ind in range(26,28):
        l_cat[ind] = 'phone games'
    for ind in range(28,32):
        l_cat[ind] = 'CD games'
    for ind in range(32,37):
        l_cat[ind] = 'Card'
    for ind in range(37,43):
        l_cat[ind] = 'Movie'
    for ind in range(43,55):
        l_cat[ind] = 'Books'
    for ind in range(55,61):
        l_cat[ind] = 'Music'
    for ind in range(61,73):
        l_cat[ind] = 'Gifts'
    for ind in range(73,79):
        l_cat[ind] = 'Soft'
    for ind in range(79,81):
        l_cat[ind] = 'Office'
    for ind in range(81,83):
        l_cat[ind] = 'Clean'
    l_cat[83] = 'Elements of a food'

    lb = preprocessing.LabelEncoder()
    item_cat['item_category_id_fix'] = lb.fit_transform(l_cat)
    item_cat['item_category_name_fix'] = l_cat
    train_test_set = train_test_set.merge(item_cat[['item_category_id', 'item_category_id_fix']], on = 'item_category_id', how = 'left')
    _ = train_test_set.drop(['item_category_id'],axis=1, inplace=True)
    train_test_set.rename(columns = {'item_category_id_fix':'item_category_id'}, inplace = True)

    _ = item_cat.drop(['item_category_id'],axis=1, inplace=True)
    _ = item_cat.drop(['item_category_name'],axis=1, inplace=True)

    item_cat.rename(columns = {'item_category_id_fix':'item_category_id'}, inplace = True)
    item_cat.rename(columns = {'item_category_name_fix':'item_category_name'}, inplace = True)
    item_cat = item_cat.drop_duplicates()
    item_cat.index = np.arange(0, len(item_cat))

In [20]:
gc.collect()

20

In [21]:
if kernel_with_output:
    if enable_feature_idea[0]:
        for diff in tqdm(lookback_range):
            feature_name = 'prev_shopitem_sales_' + str(diff)
            trainset2 = train_test_set.copy()
            trainset2.loc[:, 'date_block_num'] += diff
            trainset2.rename(columns={'item_cnt_month': feature_name}, inplace=True)
            train_test_set = train_test_set.merge(trainset2[['shop_id', 'item_id', 'date_block_num', feature_name]], on = ['shop_id', 'item_id', 'date_block_num'], how = 'left')
            train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
            new_features.append(feature_name)
    train_test_set.head(3)

100%|██████████| 12/12 [02:12<00:00, 11.06s/it]


In [22]:
del trainset2

In [23]:
gc.collect()

40

In [24]:
if kernel_with_output:
    if enable_feature_idea[1]:
        groups = train_test_set.groupby(by = ['item_id', 'date_block_num'])
        for diff in tqdm(lookback_range):
            feature_name = 'prev_item_sales_' + str(diff)
            result = groups.agg({'item_cnt_month':'mean'})
            result = result.reset_index()
            result.loc[:, 'date_block_num'] += diff
            result.rename(columns={'item_cnt_month': feature_name}, inplace=True)
            train_test_set = train_test_set.merge(result, on = ['item_id', 'date_block_num'], how = 'left')
            train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
            new_features.append(feature_name)        
    train_test_set.head(3)

100%|██████████| 12/12 [00:37<00:00,  3.10s/it]


In [25]:
del result
del groups

In [26]:
gc.collect()

40

In [27]:
#if kernel_with_output:
    #if enable_feature_idea[3]:
        #groups = train_test_set.groupby(by = ['shop_id', 'item_id', 'date_block_num'])
        #for diff in tqdm(lookback_range):
            ##feature_name = 'prev_shopitem_price_' + str(diff)
            #result = groups.agg({'item_price':'mean'})
            #result = result.reset_index()
            #result.loc[:, 'date_block_num'] += diff
            #result.rename(columns={'item_price': feature_name}, inplace=True)
            #train_test_set = train_test_set.merge(result, on = ['shop_id', 'item_id', 'date_block_num'], how = 'left')
            #train_test_set[feature_name] = train_test_set[feature_name]
            #new_features.append(feature_name)        
    #train_test_set.head(3)

In [28]:
#del result
#del groups
#gc.collect()

In [29]:


#del groups
#gc.collect()
#del item_cat
#gc.collect()
##del l_cat 
#gc.collect()

In [30]:
gc.collect()

80

In [31]:
gc.collect()

20

In [32]:
if kernel_with_output:
    items_subset = items[['item_id', 'item_name']]
    feature_count = 25
    tfidf = TfidfVectorizer(max_features=feature_count)
    items_df_item_name_text_features = pd.DataFrame(tfidf.fit_transform(items_subset['item_name']).toarray())

    cols = items_df_item_name_text_features.columns
    for i in range(feature_count):
        feature_name = 'item_name_tfidf_' + str(i)
        items_subset[feature_name] = items_df_item_name_text_features[cols[i]]
        new_features.append(feature_name)

    items_subset.drop('item_name', axis = 1, inplace = True)
    train_test_set = train_test_set.merge(items_subset, on = 'item_id', how = 'left')
    train_test_set.head()
    

In [33]:
del items_subset
gc.collect()
del items_df_item_name_text_features
gc.collect()
del tfidf
gc.collect()
del feature_count
gc.collect()



0

In [34]:
new_features

['prev_shopitem_sales_1',
 'prev_shopitem_sales_2',
 'prev_shopitem_sales_3',
 'prev_shopitem_sales_4',
 'prev_shopitem_sales_5',
 'prev_shopitem_sales_6',
 'prev_shopitem_sales_7',
 'prev_shopitem_sales_8',
 'prev_shopitem_sales_9',
 'prev_shopitem_sales_10',
 'prev_shopitem_sales_11',
 'prev_shopitem_sales_12',
 'prev_item_sales_1',
 'prev_item_sales_2',
 'prev_item_sales_3',
 'prev_item_sales_4',
 'prev_item_sales_5',
 'prev_item_sales_6',
 'prev_item_sales_7',
 'prev_item_sales_8',
 'prev_item_sales_9',
 'prev_item_sales_10',
 'prev_item_sales_11',
 'prev_item_sales_12',
 'item_name_tfidf_0',
 'item_name_tfidf_1',
 'item_name_tfidf_2',
 'item_name_tfidf_3',
 'item_name_tfidf_4',
 'item_name_tfidf_5',
 'item_name_tfidf_6',
 'item_name_tfidf_7',
 'item_name_tfidf_8',
 'item_name_tfidf_9',
 'item_name_tfidf_10',
 'item_name_tfidf_11',
 'item_name_tfidf_12',
 'item_name_tfidf_13',
 'item_name_tfidf_14',
 'item_name_tfidf_15',
 'item_name_tfidf_16',
 'item_name_tfidf_17',
 'item_name_tf

In [35]:


gc.collect()

40

In [36]:
gc.collect()

20

In [37]:
gc.collect()

20

In [38]:
train_test_set= reduce_mem_usage(train_test_set)

Mem. usage decreased to 1241.67 Mb (73.9% reduction)


In [39]:
#train_test_set.to_csv('train_test_set.csv')

In [40]:
train_test_set.describe

<bound method NDFrame.describe of           shop_id  item_id  date_block_num  item_price  item_cnt_month  \
0              59    22154               0       999.0             1.0   
1              25    22154               0       999.0             5.0   
2              24    22154               0       999.0             1.0   
3              23    22154               0         NaN             0.0   
4              19    22154               0         NaN             0.0   
...           ...      ...             ...         ...             ...   
11128045       45    18454              34         NaN            -1.0   
11128046       45    16188              34         NaN            -1.0   
11128047       45    15757              34         NaN            -1.0   
11128048       45    19648              34         NaN            -1.0   
11128049       45      969              34         NaN            -1.0   

          item_category_id  prev_shopitem_sales_1  prev_shopitem_sales_2  \
0

In [41]:
if kernel_with_output:
    current = time.time()

    baseline_features = ['shop_id', 'item_id', 'item_category_id', 'date_block_num'] +  new_features + ['item_cnt_month']

    # Clipping to range 0-20
    train_test_set['item_cnt_month'] = train_test_set.item_cnt_month.fillna(0).clip(0,20)

    # train: want rows with date_block_num from 0 to 31
    train_time_range_lo = (train_test_set['date_block_num'] >= 0)
    train_time_range_hi =  (train_test_set['date_block_num'] <= 32)

    # val: want rows with date_block_num from 22
    validation_time =  (train_test_set['date_block_num'] == 33)

    # test: want rows with date_block_num from 34
    test_time =  (train_test_set['date_block_num'] == 34)


    # Retrieve rows for train set, val set, test set
    cv_trainset = train_test_set[train_time_range_lo & train_time_range_hi]
    cv_valset = train_test_set[validation_time]
    cv_trainset = cv_trainset[baseline_features]
    cv_valset = cv_valset[baseline_features]
    testset = train_test_set[test_time]
    testset = testset[baseline_features]

  

In [42]:
new_features.dtype()

AttributeError: 'list' object has no attribute 'dtype'

In [43]:
gc.collect()

20

In [44]:
cv_trainset.describe

<bound method NDFrame.describe of           shop_id  item_id  item_category_id  date_block_num  \
0              59    22154                11               0   
1              25    22154                11               0   
2              24    22154                11               0   
3              23    22154                11               0   
4              19    22154                11               0   
...           ...      ...               ...             ...   
10892901       28    21937                10              32   
10892902       35    21937                10              32   
10892903       34    21937                10              32   
10892904       26    21937                10              32   
10892905       27    21937                10              32   

          prev_shopitem_sales_1  prev_shopitem_sales_2  prev_shopitem_sales_3  \
0                           0.0                    0.0                    0.0   
1                           0.0    

In [45]:
del train_test_set

In [46]:
gc.collect()

60

In [47]:
del train_time_range_hi

In [48]:
gc.collect()

40

In [49]:
if kernel_with_output:
    # Prepare numpy arrays for training/val/test
    cv_trainset_vals = cv_trainset.values.astype(int)
    trainx = cv_trainset_vals[:, 0:len(baseline_features) - 1]
    trainy = cv_trainset_vals[:, len(baseline_features) - 1]

    cv_valset_vals = cv_valset.values.astype(int)
    valx = cv_valset_vals[:, 0:len(baseline_features) - 1]
    valy = cv_valset_vals[:, len(baseline_features) - 1]

    testset_vals = testset.values.astype(int)
    testx = testset_vals[:, 0:len(baseline_features) - 1]
    print('Fitting...')
   
    

Fitting...


In [50]:
gc.collect()

40

In [51]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l2', 'auc'],
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "n_estimators": 1000
}

In [52]:
gc.collect()

40

In [53]:
if kernel_with_output:    
    #model = xgb.XGBRegressor(max_depth = 11, min_child_weight=0.5, subsample = 1, eta = 0.3, num_round = 1000, seed = 1, nthread = 16)
    model = lgb.LGBMRegressor(**hyper_params)

    model.fit(trainx, trainy, eval_metric='rmse')
    print("Done for god sake")


    

Done for god sake


In [54]:
if kernel_with_output:     
    preds = model.predict(valx)
    # Clipping to range 0-20
    preds = np.clip(preds, 0,20)
    print('val set rmse: ', sqrt(mean_squared_error(valy, preds)))

    preds = model.predict(testx)
    # Clipping to range 0-20
    preds = np.clip(preds, 0,20)
    df = pd.DataFrame(preds, columns = ['item_cnt_month'])
    df['ID'] = df.index
    df = df.set_index('ID')
    df.to_csv('test_preds.csv')
    print('test predictions written to file')

    end = time.time()
    diff = end - current
    print('Took ' + str(int(diff)) + ' seconds to train and predict val, test set')

val set rmse:  0.9831495612502951
test predictions written to file
Took 1078 seconds to train and predict val, test set
